In [1]:
!pip install adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.5 MB 12.7 MB/s 
     |████████████████████████████████| 895 kB 43.1 MB/s 
     |████████████████████████████████| 3.3 MB 50.5 MB/s 
     |████████████████████████████████| 50 kB 8.6 MB/s 
     |████████████████████████████████| 264 kB 13.5 MB/s 
     |████████████████████████████████| 119 kB 61.8 MB/s 
     |████████████████████████████████| 243 kB 59.3 MB/s 


In [2]:
cd drive/MyDrive/CAMemBERT2/

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import AutoModelWithHeads,AutoConfig,Trainer,TrainingArguments,EarlyStoppingCallback
import numpy as np
import pdb
from sklearn.metrics import cohen_kappa_score,f1_score,accuracy_score,fbeta_score
from scipy.stats import spearmanr
from trainer_utils import get_dataset
import torch
import pandas as pd
from transformers.adapters.composition import Fuse

dataset_obj,dataset_dict = get_dataset(['ged'],max_length=512)
# prepare for model
dataset_dict = dataset_dict.remove_columns(['dataset','scores'])


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Fa

...

  0%|          | 0/266 [00:00<?, ?ex/s]

  0%|          | 0/243 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [5]:
config_ = AutoConfig.from_pretrained('distilroberta-base',num_labels=2)
model_ = AutoModelWithHeads.from_pretrained('distilroberta-base',config=config_)
model_.load_adapter('/content/drive/MyDrive/CAMemBERT2/adapters/ner', load_as="ner", with_head=False)
adapter_setup_ = Fuse('ner')
model_.add_fusion(adapter_setup_)
model_.add_tagging_head(
            'ged_fuse',
            num_labels=2,
            overwrite_ok=True
        )
model_.set_active_adapters(adapter_setup_)
model_.train_fusion(adapter_setup_)
class_weights = dataset_obj.get_weights()
setattr(model_,'class_weights',class_weights)

def compute_metrics(p):
    logits, labels = p.predictions.flatten() , p.label_ids.flatten()
    mask = labels!=-100
    logits , labels = logits[mask] , labels[mask]
    f1 , acc , f0_5 = f1_score(logits,labels) , accuracy_score(logits,labels) , fbeta_score(logits,labels,beta=0.5)
    return {
            "f1": f1,
            "acc": acc,
            "f0_5":f0_5,
            }

class GedWordLevelTrainer(Trainer):

    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)

    def compute_loss(self,model,inputs,return_outputs=False):
        logits = model(**inputs).logits
        labels = inputs['labels']
        # Only keep active parts of the loss
        active_loss = inputs['attention_mask'].view(-1) == 1
        active_logits = logits.view(-1, model.config.num_labels)
        active_labels = torch.where(active_loss, labels.view(-1), torch.tensor(self.loss_fct.ignore_index).type_as(labels))
        non_padded_mask = active_labels!=-100
        loss = self.loss_fct(active_logits[non_padded_mask], active_labels[non_padded_mask])
        active_preds_mask = torch.logical_and(active_loss,non_padded_mask)
        return (loss,(loss,torch.argmax(logits,2))) if return_outputs else loss


lr,bs = 1e-4,8
label_names = ['labels']
args = TrainingArguments(
        output_dir = '/content',
        save_total_limit = 1,
        evaluation_strategy = "epoch",
        learning_rate = lr,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        remove_unused_columns=False,
        num_train_epochs = 15,
        weight_decay = 0.01,
        save_strategy = 'epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f0_5',
        label_names=label_names
    )

trainer = GedWordLevelTrainer(
    model=model_,
    args=args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['dev'],
    compute_metrics=compute_metrics
    )

early_stopping = EarlyStoppingCallback(early_stopping_patience = 3)
trainer.add_callback(early_stopping)
trainer.train()
trainer.evaluate(dataset_dict['test'])
pd.DataFrame(trainer.state.log_history).to_csv(f'results/ged_fuse_ner_adapters_distilroberta_lr{lr}_bs{bs}_norm.csv')

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file h

Epoch,Training Loss,Validation Loss,F1,Acc,F0 5
1,No log,0.625581,0.306394,0.453616,0.505876
2,No log,0.596494,0.426471,0.818980,0.466948
3,No log,0.638561,0.428218,0.847739,0.424647
4,0.607500,0.659192,0.416210,0.856332,0.393267


***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
Saving model checkpoint to /content/checkpoint-133
Configuration saved in /content/checkpoint-133/ner/adapter_config.json
Module weights saved in /content/checkpoint-133/ner/pytorch_adapter.bin
Configuration saved in /content/checkpoint-133/ner/adapter_fusion_config.json
Module weights saved in /content/checkpoint-133/ner/pytorch_model_adapter_fusion.bin
***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
Saving model checkpoint to /content/checkpoint-266
Configuration saved in /content/checkpoint-266/ner/adapter_config.json
Module weights saved in /content/checkpoint-266/ner/pytorch_adapter.bin
Configuration saved in /content/checkpoint-266/ner/adapter_fusion_config.json
Module weights saved in /content/checkpoint-266/ner/pytorch_model_adapter_fusion.bin
Deleting older checkpoint [/content/checkpoint-532] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
S